# Model Interpretation
## DSC-6135: Introduction to Machine Learning and Computational Statistics

### Load necessary libraries

In [1]:
# as usual, let us load all the necessary libraries
import numpy as np  # numerical computation with arrays
import pandas as pd # library to manipulate datasets using dataframes
import scipy as sp  # statistical library

# below sklearn libraries for different models
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
# plot 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### Helper functions

In [53]:
# function to generate random splits
def split_data(X, y ,seed=29):
    ''' Function to split  randomly your dataset in train and test '''
    N,D = X.shape
    test_size=0.4
    np.random.seed(seed)
    permuted_idxs = np.random.permutation(N)
    N_train = int(np.floor(N*(1-test_size)))
    train_idxs = permuted_idxs[:N_train]
    test_idxs = permuted_idxs[N_train:]
    X_train = X[train_idxs,:]
    X_test = X[test_idxs,:]
    y_train = y[train_idxs]
    y_test = y[test_idxs]
    return X_train, X_test, y_train, y_test

# function to print feature importances (size of coefficients)
def print_sorted_feat_importance(weights, colnames):
    feature_weights = pd.Series(weights, index=colnames)
    feature_weights_sorted = feature_weights.apply(np.abs).sort_values(ascending=False)

    for name in feature_weights_sorted.index:
         print('{:23s}: {:.4f}'.format(name, feature_weights[name]))
            
# function to plot confidence intervals
def plot_confidence_intervals(bootstrap_weights, ax):
    rows = bootstrap_weights.shape[0]
    columns = bootstrap_weights.shape[1]
    bootstrap_weights = bootstrap_weights.reshape((rows, columns))
    ax.set_title("Variation in weight across variations in train set")
    ax.boxplot(bootstrap_weights)
    ax.set_ylabel('Weight values')
    ax.set_xlabel('Features')
    ax.set_xticklabels(X_colnames, rotation=90)
    return ax

# function to plot predictive intervals
def plot_predictive_intervals(bootstrap_predictions, labels, ax):
    rows = bootstrap_predictions.shape[0]
    columns = bootstrap_predictions.shape[1]
    bootstrap_predictions = bootstrap_predictions.reshape((rows, columns))
    ax.set_title("Variation in prediction across variations in train set")
    ax.boxplot(bootstrap_predictions)
    ax.set_ylabel('Predicted Sales')
    ax.set_xlabel('Companies')
    ax.set_xticklabels(labels, rotation=90)
    return ax

---

## Application: Predicting total sales from marketing strategy

In this exercise, you are asked to build a machine learning model to predict the total sales of a company based on their marketing strategy (how much money they invest in advertisement and in which venues they choose to advertise) as well as a few additional features of the company.

The goal is not only to predict how much sales a company should expect, the goal is to also make a recommendation to the company on how they should adjust their marketing strategy or internal operations (e.g. you might recommend, based on your model, that the company invests more in social media marketing).

#### Load the dataset and examine it

In [10]:
# load the data using pandas
X_df = pd.read_csv('X.csv')
y_df = pd.read_csv('y.csv')

In [11]:
# print the first rows of the marketing strategies data
X_df.head()

,electricity (mega W),water (10^3 m3),bus_stops ($),sport_events ($),social_media ($),TV ($),radio ($),newspaper ($)
0,2.375463,2.042122,25.455210,47.984935,15.775558,230.1,37.8,69.2
1,2.087293,1.237373,5.730926,13.183083,23.851138,44.5,39.3,45.1
2,2.333545,0.558662,7.421910,6.081495,42.850472,17.2,45.9,69.3
3,1.219449,0.380298,9.577026,20.418795,29.425470,151.5,41.3,58.5
4,1.943312,2.126781,8.087480,16.300197,34.711596,180.8,10.8,58.4


In [12]:
# print the first rows of the outcomes (total sales)
y_df.head()

,sales (10^3 units)
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


#### Split the data into training and testing

In [13]:
# transform the data into numpy arrays
X = X_df.values
y = y_df.values

# store the names of the columns
X_colnames = X_df.columns.values
y_colnames = y_df.columns.values

In [31]:
# split data into train and test
X_train, X_test, y_train, y_test = split_data(X,y)
# name the companies in the test set
indices = ['company %d' % d for d in range(1,X_test.shape[0]+1)]
X_test_df = pd.DataFrame(X_test, columns=X_colnames, index=indices)

#### Train a linear regression model to predict the total sales from the marketing strategy data

In [14]:
# train linear model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

#### Evaluate the model on the training and testing datasets

In [15]:
# evaluate on train and test set
score_train = linear_regressor.score(X_train,y_train)
score_test = linear_regressor.score(X_test,y_test)
print('Accuracy on train set: %.2f' % score_train)
print('Accuracy on test set: %.2f' % score_test)

Accuracy on train set: 0.90
Accuracy on test set: 0.88


This model seems quite accurate! We would now like to know which feature is the most predictive, and use that information to increase sales.

---

## Exercise 1. Interpreting a Linear Regression Model

Often, we are not just interested in obtaining a prediction from a model, we are also interested in why the model has made that prediction.

For a linear regression model, $y = w_0 + w_1x_1 + w_2x_2 + \ldots + w_Dx_D$, looking at the coefficients can help us determine which feature was an important factor in the prediction.

The coefficients of `sklearn`'s linear regression model is stored in the model's `.coef_` parameter.

Based on the values of the regression coefficients, which feature do you think has the most effect on the total sales? Can you hypothesize why?

In [30]:
# # get the coefficients for the linear regression model
# weights = linear_regressor.coef_[0]
# # print the coefficients in descending order (by absolute value)
# print_sorted_feat_importance(weights, X_colnames)

## Exercise 2. Constructing Confident Intervals for Regression Coefficients

Before we start making marketing recommendations based on the regression coefficients, let's determine if these coefficients are significant. That is, are the coefficients describing a rule that is particular only to our training data and cannot generalize to new data?

To do this, we change the training data slightly (by bootstrapping) and see how much our regression coefficients change.

Based on the following box plot of the regression coefficient for models fitted on 5 bootstrapped training data sets, which feature do you think has the most effect on the total sales? Is this answer different from your answer in Exercise 1?

In [29]:
# ### we fit 5 regression models on 5 bootstrapped samples of our training data
# # number of bootstrap samples to make
# n_bootstrap = 5
# # a list to store the weights of each regression model
# bootstrap_weights = []
# for n in range(n_bootstrap):
#     # create new training data
#     X_train, _, y_train, _ = split_data(X, y, seed=n)
#     # train linear regressor model
#     linear_regressor.fit(X_train,y_train)
#     # save the regression weights
#     bootstrap_weights.append(linear_regressor.coef_[0])
    
# bootstrap_weights = np.array(bootstrap_weights)

# # make a box plot for the regression weights
# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# plot_confidence_intervals(bootstrap_weights, ax)
# plt.show()

## Exercise 3. Constructing the Predictive Interval of a Regression Model

In test data, company 30 is **Coca-cola** and company 29 is **Inyange**.

Using your linear regression model predict which company will have a higher total sales based on their marketing strategy.

In [49]:
# X_test.head()

In [56]:
# # predict on the test data
# y_predict = linear_regressor.predict(X_test)
# # print the total sales of company 29 and company 30
# print('Predicted sales for Coca-cola: %.2f (thousand units)' % y_predict[29])
# print('Predicted sales for Inyange: %.2f (thousand units)' % y_predict[30])

Construct the predictive interval for each predicted total sales. Based on these intervals, how confident are you in your predictions?

In [57]:
# ### we fit 1000 regression models on 1000 bootstrapped samples of our training data
# n_bootstrap = 1000
# # make a list to store the predicted sales
# bootstrap_y_pred = []
# for n in range(n_bootstrap):
#     # make new data
#     X_train, _, y_train, _ = split_data(X, y, seed=n)
#     # train linear regression model
#     linear_regressor.fit(X_train,y_train)
#     # predict on company 30 and 31
#     y_predict = linear_regressor.predict(X_test[29:31])
#     # add the prediction to the list
#     bootstrap_y_pred.append(y_predict)
# # convert the list into an array
# bootstrap_y_pred = np.array(bootstrap_y_pred)

# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# plot_predictive_intervals(bootstrap_y_pred, ['Coca-cola', 'Inyange'], ax)
# plt.show()